## Implementação Prática
Bem-vindo(a) ao Mapa Rodoviário do Interior de São Paulo, nosso objetivo é identificar o menor trajeto entre as cidades do nosso mapa que você informar utilizando o algoritmo de busca que você indicar.

<div align="center">
    <img src="./src/img/mapaCidades.png" alt="Mapa Rodoviário" />
</div> 

In [22]:
# Importação das bibliotecas necessárias para execução do algoritmo
from collections import deque
# Importação das bibliotecas necessárias para exibição do diagrama
import base64 
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Grafo do mapa rodoviário
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba',30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

# Associação entre os nomes das cidades e os vértices do grafo (para exibição)
nome_das_cidades = {'Piracicaba': 'Piracicaba', 'Americana': 'Americana', 'Sumaré': 'Sumare', 
                 'Campinas': 'Campinas', 'Monte Mor': 'MonteMor', 'Indaiatuba': 'Indaiatuba',
                 'Salto': 'Salto', 'Capivari': 'Capivari', 'Tietê': 'Tiete', 'Paulínea': 'Paulinea',
                 'Porto Feliz': 'PortoFeliz', 'Tatuí': 'Tatui', 'Itú': 'Itu', 'Sorocaba': 'Sorocaba',
                 'Boituva': 'Boituva'}

# Algoritmo de busca em largura (BFS)
def algoritmoBFS(grafo, origem, destino):
    cor = {}                # Armazena a cor de cada nó (BRANCO, CINZA, PRETO)
    distancia = {}          # Armazena a distância de origem até o nó
    predecessor = {}        # Armazena o predecessor (pai) de cada nó no caminho
    
    # Inicializa todos os vértices
    for vertice in grafo:  
        cor[vertice] = 'BRANCO'
        distancia[vertice] = float('inf')
        predecessor[vertice] = None
    
    # Inicializa a origem 
    cor[origem] = 'CINZA'           # A origem é marcada como cinza (visitada)
    distancia[origem] = 0           # Distância até a origem é 0
    predecessor[origem] = None      # A origem não tem predecessor
    
    # Inicializa a fila com a origem
    fila = deque([origem])
    
    # Executa a busca a partir da origem
    while fila:
        vertice_atual = fila.popleft()      # Remove o primeiro elemento da fila para ser explorado
        
        # Processa os vizinhos do nó atual
        for vertice_vizinho, custo in grafo[vertice_atual]:
            # Se o vizinho ainda não foi visitado (CINZA) ou explorado (PRETO)
            if cor[vertice_vizinho] == 'BRANCO':                
                cor[vertice_vizinho] = 'CINZA'                  # Marca como visitado
                distancia[vertice_vizinho] = distancia[vertice_atual] + custo  # Soma a distância real
                predecessor[vertice_vizinho] = vertice_atual    # Adiciona o vértice atual como predecessor da próxima iteração           
                fila.append(vertice_vizinho)                    # Enfileira o vizinho
        
        cor[vertice_atual] = 'PRETO'        # Marca o nó atual como explorado
    
    # Reconstrução do caminho do destino até a origem
    caminho = []

    # Verifica se o destino foi alcançado
    if cor[destino] == 'PRETO':  
        # Reconstrói o caminho do destino até a origem
        while destino is not None:
            caminho.append(destino)
            destino = predecessor[destino]            
        caminho.reverse()   

    # Gera o código Mermaid    
    mermaid_diagram = gerar_mermaid_flowchart(caminho, distancia, nome_das_cidades)
    # Retorna a distância até o destino
    distancia = distancia[caminho[-1]]  
    
    # Retorna o caminho e a distância total
    return caminho, distancia, mermaid_diagram

# Função auxiliar para visitar os vértices do algoritmo de busca em profundidade
def visitaDFS(grafo, vertice_atual, cor, distancia, predecessor, tempo):
    cor[vertice_atual] = 'CINZA'            # Marca o vértice como cinza (visitado)
    distancia[vertice_atual] = tempo        # Atualiza a distância do vértice

    # Processa os vizinhos do vértice atual
    for vertice_vizinho, custo in grafo[vertice_atual]:
        if cor[vertice_vizinho] == 'BRANCO':
            distancia[vertice_vizinho] = distancia[vertice_atual] + custo           # Soma a distância real
            predecessor[vertice_vizinho] = vertice_atual      
            tempo += custo                                                          # Acumula o custo da aresta
            tempo = visitaDFS(grafo, vertice_vizinho, cor, distancia, predecessor, tempo)

    cor[vertice_atual] = 'PRETO'    # Marca o vértice como explorado
    return tempo                    # Retorna o tempo atualizado

# Algoritmo de busca em profundidade (DFS)
def algoritmoDFS(grafo, origem, destino):
    cor = {}                # Armazena a cor de cada nó (BRANCO, CINZA, PRETO)
    distancia = {}          # Armazena a distância de origem até o nó
    predecessor = {}        # Armazena o predecessor (pai) de cada nó no caminho
    tempo = 0               # Contador de tempo para a busca em profundidade

    # Inicializa a origem
    cor[origem] = 'BRANCO'
    distancia[origem] = 0
    predecessor[origem] = None
            
    # Inicializa todos os vértices
    for vertice_atual in grafo:  
        cor[vertice_atual] = 'BRANCO'
        distancia[vertice_atual] = float('inf')
        predecessor[vertice_atual] = None

    # Executa a busca em profundidade a partir da origem informada pelo usuário
    tempo = visitaDFS(grafo, origem, cor, distancia, predecessor, tempo)
    
    # Reconstrução do caminho do destino até a origem
    caminho = []
    # Verifica se o destino foi alcançado
    if cor[destino] == 'PRETO':  
        while destino is not None:
            caminho.append(destino)
            destino = predecessor[destino]            
        caminho.reverse()  # Reverte o caminho para ir da origem ao destino

    # Gera o código Mermaid
    mermaid_diagram = gerar_mermaid_flowchart(caminho, distancia, nome_das_cidades)
    # Retorna a distância até o destino
    distancia = distancia[caminho[-1]]  
    
    # Retorna o caminho e a distância
    return caminho, distancia, mermaid_diagram

# Função para gerar o código Mermaid formatado
def gerar_mermaid_flowchart(caminho, distancia, nome_das_cidades):
    if not caminho:
        return "%% Nenhum caminho encontrado entre os vértices especificados."

    mermaid = "flowchart LR\n"
    for i in range(len(caminho) - 1):
        origem = caminho[i]
        destino = caminho[i + 1]
        distancia_entre_cidades = distancia[destino] - distancia[origem]
        mermaid += f"    {nome_das_cidades[origem]}([{origem}]) --- |{distancia_entre_cidades}| {nome_das_cidades[destino]}([{destino}])\n"

    return mermaid
  
# Função para gerar o código Mermaid formatado para o grafo completo
def gerar_mermaid_flowchart_completo(grafo, distancia, nome_das_cidades):
    mermaid = "flowchart TD\n"
    arestas_visitadas = set()  # Conjunto para armazenar as arestas já processadas

    for origem in grafo:
        for destino, custo in grafo[origem]:
            # Cria uma tupla da aresta para verificar duplicatas
            aresta = tuple(sorted([origem, destino]))

            # Só adiciona a aresta se ela ainda não foi processada
            if aresta not in arestas_visitadas:
                distancia_entre_cidades = custo
                mermaid += f"    {nome_das_cidades[origem]}([{origem}]) --- |{distancia_entre_cidades}| {nome_das_cidades[destino]}([{destino}])\n"
                arestas_visitadas.add(aresta)  # Marca a aresta como processada

    return mermaid


# Geração da árvore de decisão comleta 
# Função para gerar o Mermaid Flowchart
def gerar_mermaid_arvore_completa(raiz, arestas):
    flowchart = "flowchart TD\n"

    for origem, destino, custo in arestas:
        flowchart += f"    {nome_das_cidades[origem]}([{origem}]) -->|{custo}| {nome_das_cidades[destino]}([{destino}])\n"

    return flowchart

# Algoritmo DFS (Busca em profundidade)
def dfs(grafo, origem):
    visitados = set()
    arestas = []

    def dfs_recursivo(vertice_atual):
        visitados.add(vertice_atual)
        for vertice_vizinho, custo in grafo[vertice_atual]:
            if vertice_vizinho not in visitados:
                arestas.append((vertice_atual, vertice_vizinho, custo))
                dfs_recursivo(vertice_vizinho)

    dfs_recursivo(origem)
    return arestas

# Algoritmo BFS (Busca em largura)
def bfs(grafo, origem):
    visitados = set([origem])
    fila = deque([origem])
    arestas = []

    while fila:
        vertice_atual = fila.popleft()
        for vertice_vizinho, custo in grafo[vertice_atual]:
            if vertice_vizinho not in visitados:                
                visitados.add(vertice_vizinho)
                arestas.append((vertice_atual, vertice_vizinho, custo))
                fila.append(vertice_vizinho)
    
    return arestas

# Função para criar a árvore de decisão completa do grafo
def criar_arvore_decisao_completa(grafo, origem, algoritmo):
    if algoritmo == 'DFS':
        arestas = dfs(grafo, origem)
    elif algoritmo == 'BFS':
        arestas = bfs(grafo, origem)
    
    return gerar_mermaid_arvore_completa(origem, arestas)

# Função para exibir o diagrama Mermaid
def exibe_mermaid_flowchart(grafo):
    graphbytes = grafo.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

# Interação com o usuário
origem = input("Informe a cidade que originará seu trajeto: ")
destino = input("Informe a cidade de destino: ")
algoritmo = input("Informe o algoritmo de busca desejado\n\t(BFS ou DFS): ").strip().upper()

# Executa o algoritmo informado pelo usuário
if algoritmo == 'BFS':    
    caminho, distancia, mermaid_diagram = algoritmoBFS(grafo, origem, destino)
elif algoritmo == 'DFS':
    caminho, distancia, mermaid_diagram = algoritmoDFS(grafo, origem, destino)

# Salva o fiagrama gerado em um arquivo .mmd 
with open("src/flowchart_atividade_02.mmd", "w") as file:
    file.write(mermaid_diagram)

# Exibição do resultado
# Exibe o caminho percorrido entre as cidades como um diagrama de fluxo (flowchart)
print(f"O menor caminho identificado entre as cidades de {origem} e {destino} utilizando o algoritmo {algoritmo} é composto pelas seguintes cidades:")
exibe_mermaid_flowchart(mermaid_diagram)

# Exibe a distância total percorrida entre as cidades
print(f"A distância total a ser percorrida entre cada cidade é de: {distancia}km.\n")

# Exibe o diagrama de fluxo completo
print("Árvore de decisão completa:")
arvore_completa = criar_arvore_decisao_completa(grafo, origem, algoritmo)
exibe_mermaid_flowchart(arvore_completa)

# Exibe o caminho percorrido entre as cidades
# print("Caminho percorrido:")
# for cidade in caminho:
#    print(f"\t{cidade}")

# Salva o caminho percorrido em um arquivo .txt
# with open("caminho.txt", "w") as file:
#    for cidade in caminho:
#        file.write(f"{cidade}\n")
#    file.write(f"\nDistância total: {distancia}km\n")


O menor caminho identificado entre as cidades de Piracicaba e Porto Feliz utilizando o algoritmo BFS é composto pelas seguintes cidades:


A distância total a ser percorrida entre cada cidade é de: 65km.

Árvore de decisão completa:
